In [499]:
import pandas as pd
import numpy as np

In [500]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: ...working... done

# All requested packages already installed.



In [501]:
df= pd.read_csv("Collisions.csv")

In [502]:
df.head(20)

,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,LOCATION,EXCEPTRSNCODE,EXCEPTRSNDESC,SEVERITYCODE,SEVERITYDESC,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,INJURIES,SERIOUSINJURIES,FATALITIES,INCDATE,INCDTTM,JUNCTIONTYPE,SDOT_COLCODE,SDOT_COLDESC,INATTENTIONIND,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,-122.356511,47.517361,1,327920,329420,3856094,Matched,Intersection,34911.0,17TH AVE SW AND SW ROXBURY ST,,NaN,1,Property Damage Only Collision,Angles,2,0,0,2,0,0,0,2020/01/19 00:00:00+00,1/19/2020 9:01:00 AM,At Intersection (intersection related),11.0,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",NaN,N,Clear,Dry,Daylight,NaN,NaN,NaN,10,Entering at angle,0,0,N
1,-122.361405,47.702064,2,46200,46200,1791736,Matched,Block,NaN,HOLMAN RD NW BETWEEN 4TH AVE NW AND 3RD AVE NW,NaN,NaN,1,Property Damage Only Collision,Rear Ended,2,0,0,2,0,0,0,2005/04/11 00:00:00+00,4/11/2005 6:31:00 PM,Mid-Block (not related to intersection),14.0,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, REAR END",Y,0,Raining,Wet,Dusk,NaN,5101020.0,NaN,13,From same direction - both going straight - bo...,0,0,N
2,-122.317414,47.664028,3,1212,1212,3507861,Matched,Block,NaN,ROOSEVELT WAY NE BETWEEN NE 47TH ST AND NE 50T...,,NaN,2,Injury Collision,Head On,2,0,0,2,1,0,0,2013/03/31 00:00:00+00,3/31/2013 2:09:00 AM,Mid-Block (not related to intersection),11.0,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",NaN,N,Clear,Dry,Dark - Street Lights On,NaN,NaN,NaN,30,From opposite direction - all others,0,0,N
3,-122.318234,47.619927,4,327909,329409,EA03026,Matched,Intersection,29054.0,11TH AVE E AND E JOHN ST,,NaN,2,Injury Collision,Pedestrian,3,1,0,1,1,0,0,2020/01/06 00:00:00+00,1/6/2020 5:55:00 PM,At Intersection (intersection related),24.0,MOTOR VEHCILE STRUCK PEDESTRIAN,NaN,N,Raining,Wet,Dark - Street Lights On,NaN,NaN,NaN,0,Vehicle going straight hits pedestrian,0,0,N
4,-122.351724,47.560306,5,104900,104900,2671936,Matched,Block,NaN,WEST MARGINAL WAY SW BETWEEN SW ALASKA ST AND ...,NaN,NaN,2,Injury Collision,Other,2,0,0,1,1,0,0,2009/12/25 00:00:00+00,12/25/2009 7:00:00 PM,Mid-Block (not related to intersection),28.0,MOTOR VEHICLE RAN OFF ROAD - HIT FIXED OBJECT,NaN,0,Clear,Ice,Dark - Street Lights On,NaN,9359012.0,Y,50,Fixed object,0,0,N
5,-122.333067,47.544302,6,322205,323705,3694789,Matched,Block,NaN,1ST AV S NB OFF RP BETWEEN 1ST AV S BR NB AND ...,,NaN,1,Property Damage Only Collision,Rear Ended,2,0,0,2,0,0,0,2019/08/08 00:00:00+00,8/8/2019 12:49:00 PM,Mid-Block (not related to intersection),14.0,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, REAR END",NaN,N,Clear,Dry,Daylight,NaN,NaN,NaN,14,From same direction - both going straight - on...,0,0,N
6,-122.338543,47.616874,7,322275,323775,E949282,Matched,Block,NaN,8TH AVE BETWEEN WESTLAKE AVE AND BLANCHARD ST,,NaN,1,Property Damage Only Collision,Cycles,3,0,1,1,0,0,0,2019/08/08 00:00:00+00,8/8/2019 8:57:00 AM,Mid-Block (not related to intersection),18.0,"MOTOR VEHICLE STRUCK PEDALCYCLIST, FRONT END A...",Y,N,Overcast,Dry,Daylight,NaN,NaN,NaN,5,Vehicle Strikes Pedalcyclist,0,0,N
7,NaN,NaN,8,5721,5721,1786583,Unmatched,NaN,NaN,NaN,NEI,"Not Enough Information, or Insufficient Locati...",1,Property Damage Only Collision,Parked Car,0,0,0,2,0,0,0,2007/08/27 00:00:00+00,8/27/2007,Mid-Block (not related to intersection),13.0,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, LEFT SIDE ...",NaN,0,Clear,Dry,Daylight,NaN,3239035.0,NaN,32,One parked--one moving,0,0,N
8,-122.360959,47.571594,9,33400,33400,1076890,Matched,Block,NaN,WEST SEATTLE BR WB BETWEEN W SEATTLE BR WB OFF...,NaN,NaN,2b,Serious Injury Collision,Rear Ended,3,0,0,3,3,1,0,2005/07/06 00:00:00+00,7/6/2005,Mid-Block (not related to intersection),14.0,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, REAR END",Y,0,Clear,Dry,Daylight,NaN,5187041.0,NaN,13,From same direction - both going straight - bo...,0,0,N
9,-122.336856,47.655652,10,20400,20400,1220377,Matched,Block,NaN,N 40TH ST BETWEEN DENSMO

In [503]:
df.dtypes

X                  float64
Y                  float64
OBJECTID             int64
INCKEY               int64
COLDETKEY            int64
REPORTNO            object
STATUS              object
ADDRTYPE            object
INTKEY             float64
LOCATION            object
EXCEPTRSNCODE       object
EXCEPTRSNDESC       object
SEVERITYCODE        object
SEVERITYDESC        object
COLLISIONTYPE       object
PERSONCOUNT          int64
PEDCOUNT             int64
PEDCYLCOUNT          int64
VEHCOUNT             int64
INJURIES             int64
SERIOUSINJURIES      int64
FATALITIES           int64
INCDATE             object
INCDTTM             object
JUNCTIONTYPE        object
SDOT_COLCODE       float64
SDOT_COLDESC        object
INATTENTIONIND      object
UNDERINFL           object
WEATHER             object
ROADCOND            object
LIGHTCOND           object
PEDROWNOTGRNT       object
SDOTCOLNUM         float64
SPEEDING            object
ST_COLCODE          object
ST_COLDESC          object
S

In [504]:
df.shape

(221738, 40)

In [505]:
df["SEVERITYCODE"].value_counts()

1     137776
2      58842
0      21656
2b      3111
3        352
Name: SEVERITYCODE, dtype: int64

In [506]:
df["SEVERITYDESC"].value_counts()

Property Damage Only Collision    137776
Injury Collision                   58842
Unknown                            21657
Serious Injury Collision            3111
Fatality Collision                   352
Name: SEVERITYDESC, dtype: int64

In [507]:
df.drop(df.index[df["SEVERITYDESC"]=='Unknown'], axis = 0, inplace=True)
df.reset_index(inplace= True)
df["INATTENTIONIND"].replace({np.nan:0}, inplace =True)
df["INATTENTIONIND"].replace("Y", 1, inplace=True)


df["PEDROWNOTGRNT"].replace({np.nan:0},inplace =True)
df["PEDROWNOTGRNT"].replace("Y", 1, inplace=True)

df["UNDERINFL"].replace(np.nan, 0, inplace=True)
df["UNDERINFL"].replace('N', 0, inplace=True)
df["UNDERINFL"].replace('0', 0, inplace=True)
df["UNDERINFL"].replace('1', 1, inplace=True)
df["UNDERINFL"].replace("Y", 1, inplace=True)


df["SPEEDING"].replace({np.nan:0},inplace =True)
df["SPEEDING"].replace("Y", 1, inplace=True)

df["HITPARKEDCAR"].replace("N", 0, inplace=True)
df["HITPARKEDCAR"].replace(np.nan, 0, inplace=True)
df["HITPARKEDCAR"].replace("Y", 1, inplace=True)

print(df["SEVERITYDESC"].value_counts())
print(df["UNDERINFL"].value_counts())
print(df["PEDROWNOTGRNT"].value_counts())
print(df["INATTENTIONIND"].value_counts())
print(df["SPEEDING"].value_counts())


Property Damage Only Collision    137776
Injury Collision                   58842
Serious Injury Collision            3111
Fatality Collision                   352
Name: SEVERITYDESC, dtype: int64
0    190452
1      9629
Name: UNDERINFL, dtype: int64
0    194887
1      5194
Name: PEDROWNOTGRNT, dtype: int64
0    169893
1     30188
Name: INATTENTIONIND, dtype: int64
0    190146
1      9935
Name: SPEEDING, dtype: int64


In [508]:
print(df["SEVERITYCODE"].value_counts())
print(df["SEVERITYCODE"])
print(df["SEVERITYDESC"])

1     137776
2      58842
2b      3111
3        352
Name: SEVERITYCODE, dtype: int64
0         1
1         1
2         2
3         2
4         2
         ..
200076    2
200077    1
200078    2
200079    2
200080    2
Name: SEVERITYCODE, Length: 200081, dtype: object
0         Property Damage Only Collision
1         Property Damage Only Collision
2                       Injury Collision
3                       Injury Collision
4                       Injury Collision
                       ...              
200076                  Injury Collision
200077    Property Damage Only Collision
200078                  Injury Collision
200079                  Injury Collision
200080                  Injury Collision
Name: SEVERITYDESC, Length: 200081, dtype: object


In [509]:
for i in range(0,len(df["SEVERITYCODE"])):
    if df["SEVERITYDESC"][i] == "Serious Injury Collision":
        df["SEVERITYCODE"][i] = 3
    if df["SEVERITYDESC"][i] == "Fatality Collision":
        df["SEVERITYCODE"][i] = 4
print(df["SEVERITYCODE"].value_counts())


C:\Users\ARINDAM\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\ARINDAM\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


1    137776
2     58842
3      3111
4       352
Name: SEVERITYCODE, dtype: int64


In [510]:
df.shape

(200081, 41)

In [511]:
df["SEVERITYDESC"].value_counts()

Property Damage Only Collision    137776
Injury Collision                   58842
Serious Injury Collision            3111
Fatality Collision                   352
Name: SEVERITYDESC, dtype: int64

In [512]:
df.drop(["index","OBJECTID","STATUS","INCKEY","COLDETKEY","INTKEY","REPORTNO","EXCEPTRSNCODE","EXCEPTRSNDESC","SDOTCOLNUM","ST_COLCODE","ST_COLDESC","SDOT_COLCODE","SDOT_COLDESC","JUNCTIONTYPE"],axis= 1, inplace= True)

In [513]:
df.shape

(200081, 26)

In [514]:
df.head()

,X,Y,ADDRTYPE,LOCATION,SEVERITYCODE,SEVERITYDESC,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,INJURIES,SERIOUSINJURIES,FATALITIES,INCDATE,INCDTTM,INATTENTIONIND,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SPEEDING,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,-122.356511,47.517361,Intersection,17TH AVE SW AND SW ROXBURY ST,1,Property Damage Only Collision,Angles,2,0,0,2,0,0,0,2020/01/19 00:00:00+00,1/19/2020 9:01:00 AM,0,0,Clear,Dry,Daylight,0,0,0,0,0
1,-122.361405,47.702064,Block,HOLMAN RD NW BETWEEN 4TH AVE NW AND 3RD AVE NW,1,Property Damage Only Collision,Rear Ended,2,0,0,2,0,0,0,2005/04/11 00:00:00+00,4/11/2005 6:31:00 PM,1,0,Raining,Wet,Dusk,0,0,0,0,0
2,-122.317414,47.664028,Block,ROOSEVELT WAY NE BETWEEN NE 47TH ST AND NE 50T...,2,Injury Collision,Head On,2,0,0,2,1,0,0,2013/03/31 00:00:00+00,3/31/2013 2:09:00 AM,0,0,Clear,Dry,Dark - Street Lights On,0,0,0,0,0
3,-122.318234,47.619927,Intersection,11TH AVE E AND E JOHN ST,2,Injury Collision,Pedestrian,3,1,0,1,1,0,0,2020/01/06 00:00:00+00,1/6/2020 5:55:00 PM,0,0,Raining,Wet,Dark - Street Lights On,0,0,0,0,0
4,-122.351724,47.560306,Block,WEST MARGINAL WAY SW BETWEEN SW ALASKA ST AND ...,2,Injury Collision,Other,2,0,0,1,1,0,0,2009/12/25 00:00:00+00,12/25/2009 7:00:00 PM,0,0,Clear,Ice,Dark - Street Lights On,0,1,0,0,0


In [515]:
df.shape

(200081, 26)

In [516]:
df.dtypes

X                  float64
Y                  float64
ADDRTYPE            object
LOCATION            object
SEVERITYCODE        object
SEVERITYDESC        object
COLLISIONTYPE       object
PERSONCOUNT          int64
PEDCOUNT             int64
PEDCYLCOUNT          int64
VEHCOUNT             int64
INJURIES             int64
SERIOUSINJURIES      int64
FATALITIES           int64
INCDATE             object
INCDTTM             object
INATTENTIONIND       int64
UNDERINFL            int64
WEATHER             object
ROADCOND            object
LIGHTCOND           object
PEDROWNOTGRNT        int64
SPEEDING             int64
SEGLANEKEY           int64
CROSSWALKKEY         int64
HITPARKEDCAR         int64
dtype: object

In [517]:
pd.set_option('display.max_columns', None)
df=df.astype({"SEVERITYCODE":np.int})
df.head(10)

,X,Y,ADDRTYPE,LOCATION,SEVERITYCODE,SEVERITYDESC,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,INJURIES,SERIOUSINJURIES,FATALITIES,INCDATE,INCDTTM,INATTENTIONIND,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SPEEDING,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,-122.356511,47.517361,Intersection,17TH AVE SW AND SW ROXBURY ST,1,Property Damage Only Collision,Angles,2,0,0,2,0,0,0,2020/01/19 00:00:00+00,1/19/2020 9:01:00 AM,0,0,Clear,Dry,Daylight,0,0,0,0,0
1,-122.361405,47.702064,Block,HOLMAN RD NW BETWEEN 4TH AVE NW AND 3RD AVE NW,1,Property Damage Only Collision,Rear Ended,2,0,0,2,0,0,0,2005/04/11 00:00:00+00,4/11/2005 6:31:00 PM,1,0,Raining,Wet,Dusk,0,0,0,0,0
2,-122.317414,47.664028,Block,ROOSEVELT WAY NE BETWEEN NE 47TH ST AND NE 50T...,2,Injury Collision,Head On,2,0,0,2,1,0,0,2013/03/31 00:00:00+00,3/31/2013 2:09:00 AM,0,0,Clear,Dry,Dark - Street Lights On,0,0,0,0,0
3,-122.318234,47.619927,Intersection,11TH AVE E AND E JOHN ST,2,Injury Collision,Pedestrian,3,1,0,1,1,0,0,2020/01/06 00:00:00+00,1/6/2020 5:55:00 PM,0,0,Raining,Wet,Dark - Street Lights On,0,0,0,0,0
4,-122.351724,47.560306,Block,WEST MARGINAL WAY SW BETWEEN SW ALASKA ST AND ...,2,Injury Collision,Other,2,0,0,1,1,0,0,2009/12/25 00:00:00+00,12/25/2009 7:00:00 PM,0,0,Clear,Ice,Dark - Street Lights On,0,1,0,0,0
5,-122.333067,47.544302,Block,1ST AV S NB OFF RP BETWEEN 1ST AV S BR NB AND ...,1,Property Damage Only Collision,Rear Ended,2,0,0,2,0,0,0,2019/08/08 00:00:00+00,8/8/2019 12:49:00 PM,0,0,Clear,Dry,Daylight,0,0,0,0,0
6,-122.338543,47.616874,Block,8TH AVE BETWEEN WESTLAKE AVE AND BLANCHARD ST,1,Property Damage Only Collision,Cycles,3,0,1,1,0,0,0,2019/08/08 00:00:00+00,8/8/2019 8:57:00 AM,1,0,Overcast,Dry,Daylight,0,0,0,0,0
7,NaN,NaN,NaN,NaN,1,Property Damage Only Collision,Parked Car,0,0,0,2,0,0,0,2007/08/27 00:00:00+00,8/27/2007,0,0,Clear,Dry,Daylight,0,0,0,0,0
8,-122.360959,47.571594,Block,WEST SEATTLE BR WB BETWEEN W SEATTLE BR WB OFF...,3,Serious Injury Collision,Rear Ended,3,0,0,3,3,1,0,2005/07/06 00:00:00+00,7/6/2005,1,0,Clear,Dry,Daylight,0,0,0,0,0
9,-122.336856,47.655652,Block,N 40TH ST BETWEEN DENSMORE AVE N AND WALLINGFO...,1,Property Damage Only Collision,Rear Ended,2,0,0,2,0,0,0,2004/08/03 00:00:00+00,8/3/2004,1,0,Clear,Wet,Dusk,0,0,0,0,0


In [518]:
df.dtypes

X                  float64
Y                  float64
ADDRTYPE            object
LOCATION            object
SEVERITYCODE         int32
SEVERITYDESC        object
COLLISIONTYPE       object
PERSONCOUNT          int64
PEDCOUNT             int64
PEDCYLCOUNT          int64
VEHCOUNT             int64
INJURIES             int64
SERIOUSINJURIES      int64
FATALITIES           int64
INCDATE             object
INCDTTM             object
INATTENTIONIND       int64
UNDERINFL            int64
WEATHER             object
ROADCOND            object
LIGHTCOND           object
PEDROWNOTGRNT        int64
SPEEDING             int64
SEGLANEKEY           int64
CROSSWALKKEY         int64
HITPARKEDCAR         int64
dtype: object

In [519]:
df.reset_index(inplace= True)

In [520]:
df.columns

Index(['index', 'X', 'Y', 'ADDRTYPE', 'LOCATION', 'SEVERITYCODE',
       'SEVERITYDESC', 'COLLISIONTYPE', 'PERSONCOUNT', 'PEDCOUNT',
       'PEDCYLCOUNT', 'VEHCOUNT', 'INJURIES', 'SERIOUSINJURIES', 'FATALITIES',
       'INCDATE', 'INCDTTM', 'INATTENTIONIND', 'UNDERINFL', 'WEATHER',
       'ROADCOND', 'LIGHTCOND', 'PEDROWNOTGRNT', 'SPEEDING', 'SEGLANEKEY',
       'CROSSWALKKEY', 'HITPARKEDCAR'],
      dtype='object')

In [521]:
for i in df.columns:
    df=df.drop(df.index[df[i].isnull()== True])
df.reset_index(inplace = True)

In [522]:
df.shape

(189559, 28)

In [523]:
df.head(20)

,level_0,index,X,Y,ADDRTYPE,LOCATION,SEVERITYCODE,SEVERITYDESC,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,INJURIES,SERIOUSINJURIES,FATALITIES,INCDATE,INCDTTM,INATTENTIONIND,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SPEEDING,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,0,0,-122.356511,47.517361,Intersection,17TH AVE SW AND SW ROXBURY ST,1,Property Damage Only Collision,Angles,2,0,0,2,0,0,0,2020/01/19 00:00:00+00,1/19/2020 9:01:00 AM,0,0,Clear,Dry,Daylight,0,0,0,0,0
1,1,1,-122.361405,47.702064,Block,HOLMAN RD NW BETWEEN 4TH AVE NW AND 3RD AVE NW,1,Property Damage Only Collision,Rear Ended,2,0,0,2,0,0,0,2005/04/11 00:00:00+00,4/11/2005 6:31:00 PM,1,0,Raining,Wet,Dusk,0,0,0,0,0
2,2,2,-122.317414,47.664028,Block,ROOSEVELT WAY NE BETWEEN NE 47TH ST AND NE 50T...,2,Injury Collision,Head On,2,0,0,2,1,0,0,2013/03/31 00:00:00+00,3/31/2013 2:09:00 AM,0,0,Clear,Dry,Dark - Street Lights On,0,0,0,0,0
3,3,3,-122.318234,47.619927,Intersection,11TH AVE E AND E JOHN ST,2,Injury Collision,Pedestrian,3,1,0,1,1,0,0,2020/01/06 00:00:00+00,1/6/2020 5:55:00 PM,0,0,Raining,Wet,Dark - Street Lights On,0,0,0,0,0
4,4,4,-122.351724,47.560306,Block,WEST MARGINAL WAY SW BETWEEN SW ALASKA ST AND ...,2,Injury Collision,Other,2,0,0,1,1,0,0,2009/12/25 00:00:00+00,12/25/2009 7:00:00 PM,0,0,Clear,Ice,Dark - Street Lights On,0,1,0,0,0
5,5,5,-122.333067,47.544302,Block,1ST AV S NB OFF RP BETWEEN 1ST AV S BR NB AND ...,1,Property Damage Only Collision,Rear Ended,2,0,0,2,0,0,0,2019/08/08 00:00:00+00,8/8/2019 12:49:00 PM,0,0,Clear,Dry,Daylight,0,0,0,0,0
6,6,6,-122.338543,47.616874,Block,8TH AVE BETWEEN WESTLAKE AVE AND BLANCHARD ST,1,Property Damage Only Collision,Cycles,3,0,1,1,0,0,0,2019/08/08 00:00:00+00,8/8/2019 8:57:00 AM,1,0,Overcast,Dry,Daylight,0,0,0,0,0
7,8,8,-122.360959,47.571594,Block,WEST SEATTLE BR WB BETWEEN W SEATTLE BR WB OFF...,3,Serious Injury Collision,Rear Ended,3,0,0,3,3,1,0,2005/07/06 00:00:00+00,7/6/2005,1,0,Clear,Dry,Daylight,0,0,0,0,0
8,9,9,-122.336856,47.655652,Block,N 40TH ST BETWEEN DENSMORE AVE N AND WALLINGFO...,1,Property Damage Only Collision,Rear Ended,2,0,0,2,0,0,0,2004/08/03 00:00:00+00,8/3/2004,1,0,Clear,Wet,Dusk,0,0,0,0,0
9,10,10,-122.385859,47.581191,Block,SW ADMIRAL WAY BETWEEN 42ND AVE SW AND CALIFOR...,1,Property Damage Only Collision,Angles,2,0,0,2,0,0,0,2007/07/23 00:00:00+00,7/23/2007 4:30:00 PM,0,0,Clear,Dry,Daylight,0,0,0,0,0


In [524]:
df.isnull().sum(axis=0)

level_0            0
index              0
X                  0
Y                  0
ADDRTYPE           0
LOCATION           0
SEVERITYCODE       0
SEVERITYDESC       0
COLLISIONTYPE      0
PERSONCOUNT        0
PEDCOUNT           0
PEDCYLCOUNT        0
VEHCOUNT           0
INJURIES           0
SERIOUSINJURIES    0
FATALITIES         0
INCDATE            0
INCDTTM            0
INATTENTIONIND     0
UNDERINFL          0
WEATHER            0
ROADCOND           0
LIGHTCOND          0
PEDROWNOTGRNT      0
SPEEDING           0
SEGLANEKEY         0
CROSSWALKKEY       0
HITPARKEDCAR       0
dtype: int64

In [525]:
df.drop(["level_0","index"], axis= 1 ,inplace = True)

In [526]:
df.head(10)

,X,Y,ADDRTYPE,LOCATION,SEVERITYCODE,SEVERITYDESC,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,INJURIES,SERIOUSINJURIES,FATALITIES,INCDATE,INCDTTM,INATTENTIONIND,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SPEEDING,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,-122.356511,47.517361,Intersection,17TH AVE SW AND SW ROXBURY ST,1,Property Damage Only Collision,Angles,2,0,0,2,0,0,0,2020/01/19 00:00:00+00,1/19/2020 9:01:00 AM,0,0,Clear,Dry,Daylight,0,0,0,0,0
1,-122.361405,47.702064,Block,HOLMAN RD NW BETWEEN 4TH AVE NW AND 3RD AVE NW,1,Property Damage Only Collision,Rear Ended,2,0,0,2,0,0,0,2005/04/11 00:00:00+00,4/11/2005 6:31:00 PM,1,0,Raining,Wet,Dusk,0,0,0,0,0
2,-122.317414,47.664028,Block,ROOSEVELT WAY NE BETWEEN NE 47TH ST AND NE 50T...,2,Injury Collision,Head On,2,0,0,2,1,0,0,2013/03/31 00:00:00+00,3/31/2013 2:09:00 AM,0,0,Clear,Dry,Dark - Street Lights On,0,0,0,0,0
3,-122.318234,47.619927,Intersection,11TH AVE E AND E JOHN ST,2,Injury Collision,Pedestrian,3,1,0,1,1,0,0,2020/01/06 00:00:00+00,1/6/2020 5:55:00 PM,0,0,Raining,Wet,Dark - Street Lights On,0,0,0,0,0
4,-122.351724,47.560306,Block,WEST MARGINAL WAY SW BETWEEN SW ALASKA ST AND ...,2,Injury Collision,Other,2,0,0,1,1,0,0,2009/12/25 00:00:00+00,12/25/2009 7:00:00 PM,0,0,Clear,Ice,Dark - Street Lights On,0,1,0,0,0
5,-122.333067,47.544302,Block,1ST AV S NB OFF RP BETWEEN 1ST AV S BR NB AND ...,1,Property Damage Only Collision,Rear Ended,2,0,0,2,0,0,0,2019/08/08 00:00:00+00,8/8/2019 12:49:00 PM,0,0,Clear,Dry,Daylight,0,0,0,0,0
6,-122.338543,47.616874,Block,8TH AVE BETWEEN WESTLAKE AVE AND BLANCHARD ST,1,Property Damage Only Collision,Cycles,3,0,1,1,0,0,0,2019/08/08 00:00:00+00,8/8/2019 8:57:00 AM,1,0,Overcast,Dry,Daylight,0,0,0,0,0
7,-122.360959,47.571594,Block,WEST SEATTLE BR WB BETWEEN W SEATTLE BR WB OFF...,3,Serious Injury Collision,Rear Ended,3,0,0,3,3,1,0,2005/07/06 00:00:00+00,7/6/2005,1,0,Clear,Dry,Daylight,0,0,0,0,0
8,-122.336856,47.655652,Block,N 40TH ST BETWEEN DENSMORE AVE N AND WALLINGFO...,1,Property Damage Only Collision,Rear Ended,2,0,0,2,0,0,0,2004/08/03 00:00:00+00,8/3/2004,1,0,Clear,Wet,Dusk,0,0,0,0,0
9,-122.385859,47.581191,Block,SW ADMIRAL WAY BETWEEN 42ND AVE SW AND CALIFOR...,1,Property Damage Only Collision,Angles,2,0,0,2,0,0,0,2007/07/23 00:00:00+00,7/23/2007 4:30:00 PM,0,0,Clear,Dry,Daylight,0,0,0,0,0


In [527]:
mean_long = df["X"].mean()
mean_lat = df["Y"].mean()

print(mean_long)
print(mean_lat)

seattle_map = folium.Map(location=[mean_lat, mean_long], zoom_start=13)
seattle_map

-122.3305310565504
47.619479381609004


In [528]:
df.dtypes

X                  float64
Y                  float64
ADDRTYPE            object
LOCATION            object
SEVERITYCODE         int32
SEVERITYDESC        object
COLLISIONTYPE       object
PERSONCOUNT          int64
PEDCOUNT             int64
PEDCYLCOUNT          int64
VEHCOUNT             int64
INJURIES             int64
SERIOUSINJURIES      int64
FATALITIES           int64
INCDATE             object
INCDTTM             object
INATTENTIONIND       int64
UNDERINFL            int64
WEATHER             object
ROADCOND            object
LIGHTCOND           object
PEDROWNOTGRNT        int64
SPEEDING             int64
SEGLANEKEY           int64
CROSSWALKKEY         int64
HITPARKEDCAR         int64
dtype: object

After much cleaning the features that are left to be conditioned are "WEATHER", "ROADCOND", "LIGHTCOND". They are categorical data so need to be changed to numeric via one hot encoding. But before that the entried having "Unknown" need to be removed

In [529]:
df.drop(df.index[df["WEATHER"]=='Unknown'], axis = 0, inplace=True)
df.drop(df.index[df["ROADCOND"]=='Unknown'], axis = 0, inplace=True)
df.drop(df.index[df["LIGHTCOND"]=='Unknown'], axis = 0, inplace=True)
df.reset_index(inplace= True)

print(df["WEATHER"].value_counts())
print(df["ROADCOND"].value_counts())
print(df["LIGHTCOND"].value_counts())
print(df["SEVERITYCODE"].value_counts())

Clear                       110441
Raining                      32489
Overcast                     27244
Snowing                        831
Fog/Smog/Smoke                 547
Other                          253
Sleet/Hail/Freezing Rain       113
Blowing Sand/Dirt               43
Severe Crosswind                25
Partly Cloudy                   10
Name: WEATHER, dtype: int64
Dry               123433
Wet                46319
Ice                 1092
Snow/Slush           842
Other                102
Standing Water        99
Sand/Mud/Dirt         59
Oil                   50
Name: ROADCOND, dtype: int64
Daylight                    113856
Dark - Street Lights On      47286
Dusk                          5721
Dawn                          2449
Dark - No Street Lights       1381
Dark - Street Lights Off      1114
Other                          167
Dark - Unknown Lighting         22
Name: LIGHTCOND, dtype: int64
1    113151
2     55572
3      2945
4       328
Name: SEVERITYCODE, dtype: int64


In [533]:
dummy_variable_1 = pd.get_dummies(df["WEATHER"])
dummy_variable_2 = pd.get_dummies(df["ROADCOND"])
dummy_variable_3 = pd.get_dummies(df["LIGHTCOND"])
df=pd.concat([df, dummy_variable_1,dummy_variable_2,dummy_variable_3], axis=1)
df.drop(["WEATHER","ROADCOND", "LIGHTCOND"],axis=1, inplace= True)

In [534]:
df.head(20)

,index,X,Y,ADDRTYPE,LOCATION,SEVERITYCODE,SEVERITYDESC,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,INJURIES,SERIOUSINJURIES,FATALITIES,INCDATE,INCDTTM,INATTENTIONIND,UNDERINFL,PEDROWNOTGRNT,SPEEDING,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR,Blowing Sand/Dirt,Clear,Fog/Smog/Smoke,Other,Overcast,Partly Cloudy,Raining,Severe Crosswind,Sleet/Hail/Freezing Rain,Snowing,Dry,Ice,Oil,Other,Sand/Mud/Dirt,Snow/Slush,Standing Water,Wet,Dark - No Street Lights,Dark - Street Lights Off,Dark - Street Lights On,Dark - Unknown Lighting,Dawn,Daylight,Dusk,Other,Blowing Sand/Dirt,Clear,Fog/Smog/Smoke,Other,Overcast,Partly Cloudy,Raining,Severe Crosswind,Sleet/Hail/Freezing Rain,Snowing,Dry,Ice,Oil,Other,Sand/Mud/Dirt,Snow/Slush,Standing Water,Wet,Dark - No Street Lights,Dark - Street Lights Off,Dark - Street Lights On,Dark - Unknown Lighting,Dawn,Daylight,Dusk,Other
0,0,-122.356511,47.517361,Intersection,17TH AVE SW AND SW ROXBURY ST,1,Property Damage Only Collision,Angles,2,0,0,2,0,0,0,2020/01/19 00:00:00+00,1/19/2020 9:01:00 AM,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,1,-122.361405,47.702064,Block,HOLMAN RD NW BETWEEN 4TH AVE NW AND 3RD AVE NW,1,Property Damage Only Collision,Rear Ended,2,0,0,2,0,0,0,2005/04/11 00:00:00+00,4/11/2005 6:31:00 PM,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0
2,2,-122.317414,47.664028,Block,ROOSEVELT WAY NE BETWEEN NE 47TH ST AND NE 50T...,2,Injury Collision,Head On,2,0,0,2,1,0,0,2013/03/31 00:00:00+00,3/31/2013 2:09:00 AM,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,3,-122.318234,47.619927,Intersection,11TH AVE E AND E JOHN ST,2,Injury Collision,Pedestrian,3,1,0,1,1,0,0,2020/01/06 00:00:00+00,1/6/2020 5:55:00 PM,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0
4,4,-122.351724,47.560306,Block,WEST MARGINAL WAY SW BETWEEN SW ALASKA ST AND ...,2,Injury Collision,Other,2,0,0,1,1,0,0,2009/12/25 00:00:00+00,12/25/2009 7:00:00 PM,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
5,5,-122.333067,47.544302,Block,1ST AV S NB OFF RP BETWEEN 1ST AV S BR NB AND ...,1,Property Damage Only Collision,Rear Ended,2,0,0,2,0,0,0,2019/08/08 00:00:00+00,8/8/2019 12:49:00 PM,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
6,6,-122.338543,47.616874,Block,8TH AVE BETWEEN WESTLAKE AVE AND BLANCHARD ST,1,Property Damage Only Collision,Cycles,3,0,1,1,0,0,0,2019/08/08 00:00:00+00,8/8/2019 8:57:00 AM,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
7,7,-122.360959,47.571594,Block,WEST SEATTLE BR WB BETWEEN W SEATTLE BR WB OFF...,3,Serious Injury Collision,Rear Ended,3,0,0,3,3,1,0,2005/07/06 00:00:00+00,7/6/2005,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
8,8,-122.336856,47.655652,Block,N 40TH ST BETWEEN DENSMORE AVE N AND WALLINGFO...,1,Property Damage Only Collision,Rear Ended,2,0,0,2,0,0,0,2004/08/03 00:00:00+00,8/3/2004,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0
9,9,-122.385859,47.581191,Block,SW ADMIRAL WAY BETWEEN 42ND AVE SW AND CALIFOR...,1,Property Damage Only Collision,Angles,2,0,0,2,0,0,0,2007/07/23 00:00:00+00,7/23/2007 4:30:00 PM,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [544]:

pd.set_option('display.max_rows',80 )
df.dtypes

index                         int64
X                           float64
Y                           float64
ADDRTYPE                     object
LOCATION                     object
SEVERITYCODE                  int32
SEVERITYDESC                 object
COLLISIONTYPE                object
PERSONCOUNT                   int64
PEDCOUNT                      int64
PEDCYLCOUNT                   int64
VEHCOUNT                      int64
INJURIES                      int64
SERIOUSINJURIES               int64
FATALITIES                    int64
INCDATE                      object
INCDTTM                      object
INATTENTIONIND                int64
UNDERINFL                     int64
PEDROWNOTGRNT                 int64
SPEEDING                      int64
SEGLANEKEY                    int64
CROSSWALKKEY                  int64
HITPARKEDCAR                  int64
Blowing Sand/Dirt             uint8
Clear                         uint8
Fog/Smog/Smoke                uint8
Other                       